# 作業 : (Kaggle)房價預測

# [作業目標]
- 試著模仿範例寫法, 在房價預測中, 觀察去除離群值的影響

# [作業重點]
- 觀察將極端值以上下限值取代, 對於分布與迴歸分數的影響 (In[5], Out[5])
- 觀察將極端值資料直接刪除, 對於分布與迴歸分數的影響 (In[6], Out[6])

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

data_path = './D10_DATA/'
df_train = pd.read_csv(data_path + 'house_train.csv.gz')

train_Y = np.log1p(df_train['SalePrice'])
df = df_train.drop(['Id', 'SalePrice'] , axis=1)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: './D10_DATA/house_train.csv.gz'

In [ ]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

In [ ]:
# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
train_num = train_Y.shape[0]
print(train_num, train_Y.shape)
df.head()

# 作業1
* 試著限制 '1樓地板面積(平方英尺)' (1stFlrSF) 欄位的上下限, 看看能否再進一步提高分數?

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# 顯示 1stFlrSF 與目標值的散佈圖
sns.regplot(x = df['1stFlrSF'], y=train_Y)
plt.show()

# 做線性迴歸, 觀察分數
train_X = MMEncoder.fit_transform(df)
estimator = LinearRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

In [ ]:
df['1stFlrSF'].describe()

In [ ]:
# 將 1stFlrSF 限制在你覺得適合的範圍內, 調整離群值
"""
Your Code Here
"""
df['1stFlrSF'] = df['1stFlrSF'].clip(600, 2000)
sns.regplot(df['1stFlrSF'], train_Y)
plt.show()

# 做線性迴歸, 觀察分數
train_X = MMEncoder.fit_transform(df)
estimator = LinearRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

# 作業2
* 續前題, 去除離群值有兩類方式 :  捨棄離群值(刪除離群的資料) 以及調整離群值,  
請試著用同樣的上下限, 改為 '捨棄離群值' 的方法, 看看結果會變好還是變差? 並試著解釋原因。

In [ ]:
# 將 1stFlrSF 限制在你覺得適合的範圍內, 捨棄離群值
"""
Your Code Here
"""
remain_index = (df['1stFlrSF']>600) & (df['1stFlrSF']<2000)
df = df[remain_index]
train_Y = train_Y[remain_index]
sns.regplot(df['1stFlrSF'], train_Y)
plt.show()

# 做線性迴歸, 觀察分數
train_X = MMEncoder.fit_transform(df)
estimator = LinearRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

#### 捨棄離群值的方法, 結果變好還是變差?
- Asn:變好。
    - 原因:原本'1stFlrSF'中低於600的值對應'SalePrice'的值也是較低的，但我們卻把這些outlier values平移到600上，如此一來會破壞原本值在600的'SalePrice'。
    - 舉例來說:  
        '1stFlrSF' = [100, 200, 300, 600, 600, 630] 對應 'SalePrice' = [10, 20, 30, 60, 62, 65]  
        經過clip調整後:  
        '1stFlrSF' = [600, 600, 600, 600, 600, 630] 對應 'SalePrice' = [10, 20, 30, 60, 62, 65]  
        'SalePrice'並沒有做改變，這樣將破壞原本600附近的平均值  
    - 結論:捨棄那些強制更改的值，才會比較貼近合理的regression predict        